# Data Setup and Preprocessing Pipelines (The Professional Way)

<hr>

<center>
<div>
<img src="https://raw.githubusercontent.com/davi-moreira/2026Summer_predictive_analytics_purdue_MGMT474/main/notebooks/figures/mgmt_474_ai_logo_02-modified.png" width="200"/>
</div>
</center>

# <center><a class="tocSkip"></center>
# <center>MGMT47400 Predictive Analytics</center>
# <center>Professor: Davi Moreira </center>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/davi-moreira/2026Summer_predictive_analytics_purdue_MGMT474/blob/main/notebooks/02_preprocessing_pipelines.ipynb)

---

## Learning Objectives

By the end of this notebook, you will be able to:

1. Audit data types and fix common pandas pitfalls (strings, categories, dates)
2. Handle missing values without leaking information
3. Build a preprocessing + model Pipeline with `ColumnTransformer`
4. Separate "fit on train only" logic from evaluation logic
5. Use Gemini to draft pipeline code and then harden it (tests + comments)

---

## 1. Setup: Imports and Configuration

In [ ]:
# Install required packages (uncomment if needed)
# !pip install pandas numpy matplotlib seaborn scikit-learn --quiet

# Core imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
import warnings

# Display settings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 3)
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

# Set random seed for reproducibility
RANDOM_SEED = 474
np.random.seed(RANDOM_SEED)

print("✓ Setup complete!")
print(f"Random seed: {RANDOM_SEED}")

## 2. Load Dataset and Create Splits

In [ ]:
# Load California Housing dataset
from sklearn.datasets import fetch_california_housing

california = fetch_california_housing(as_frame=True)
df = california.frame

print(f"Dataset shape: {df.shape}")
print(f"\nFeatures: {df.columns.tolist()}")
df.head()

In [ ]:
# Separate features and target
X = df.drop(columns=['MedHouseVal'])
y = df['MedHouseVal']

# Split: 60% train, 20% validation, 20% test
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.20, random_state=RANDOM_SEED
)

X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.25, random_state=RANDOM_SEED
)

print("=== SPLIT SIZES ===")
print(f"Train: {len(X_train)} samples ({len(X_train)/len(df)*100:.1f}%)")
print(f"Validation: {len(X_val)} samples ({len(X_val)/len(df)*100:.1f}%)")
print(f"Test: {len(X_test)} samples ({len(X_test)/len(df)*100:.1f}%)")

## 3. Data Audit Report Function

Let's create a function to systematically audit our data quality.

In [ ]:
def make_data_report(df, name="Dataset"):
    """
    Generate a comprehensive data audit report.
    
    Parameters:
    -----------
    df : pd.DataFrame
        The dataframe to audit
    name : str
        Name of the dataset for reporting
    
    Returns:
    --------
    pd.DataFrame
        Report with types, missingness, and unique counts
    """
    print(f"\n{'='*60}")
    print(f"DATA AUDIT REPORT: {name}")
    print(f"{'='*60}")
    
    print(f"\nShape: {df.shape[0]:,} rows × {df.shape[1]} columns")
    
    # Build report dataframe
    report = pd.DataFrame({
        'dtype': df.dtypes,
        'missing_count': df.isnull().sum(),
        'missing_pct': (df.isnull().sum() / len(df) * 100).round(2),
        'unique_count': df.nunique(),
        'unique_pct': (df.nunique() / len(df) * 100).round(2)
    })
    
    print("\n=== COLUMN AUDIT ===")
    print(report)
    
    # Summary statistics
    print(f"\n=== SUMMARY ===")
    print(f"Total missing values: {df.isnull().sum().sum():,}")
    print(f"Columns with missing data: {(df.isnull().sum() > 0).sum()}")
    print(f"Numeric columns: {df.select_dtypes(include=[np.number]).shape[1]}")
    print(f"Non-numeric columns: {df.select_dtypes(exclude=[np.number]).shape[1]}")
    
    return report

# Run audit on training data
train_report = make_data_report(X_train, "Training Set")

## 📝 PAUSE-AND-DO Exercise 1 (10 minutes)

**Task:** Implement `make_data_report(df)` that returns types, missingness %, and unique counts.

**Instructions:**
1. The function is already implemented above
2. Run it on `X_train`, `X_val`, and `X_test`
3. Document any differences you observe between splits
4. Write your observations in the cell below

---

In [ ]:
# Run audit on all splits
val_report = make_data_report(X_val, "Validation Set")
test_report = make_data_report(X_test, "Test Set")

### YOUR OBSERVATIONS HERE:

**Observation 1:**  
[Document any differences in data types across splits]

**Observation 2:**  
[Document any missingness patterns]

**Observation 3:**  
[Document any unique value differences]

---

## 4. Preprocessing Pipeline Template

### 4.1 Identify Column Types

In [ ]:
# Identify numeric and categorical columns
numeric_features = X_train.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = X_train.select_dtypes(exclude=[np.number]).columns.tolist()

print("=== FEATURE TYPES ===")
print(f"\nNumeric features ({len(numeric_features)}):")
for feat in numeric_features:
    print(f"  - {feat}")

print(f"\nCategorical features ({len(categorical_features)}):")
if categorical_features:
    for feat in categorical_features:
        print(f"  - {feat}")
else:
    print("  (none)")

### 4.2 Build Preprocessing Pipeline

**Pipeline Design Principles:**
1. Fit transformers ONLY on training data
2. Apply the same transformation to validation and test
3. Handle numeric and categorical features separately
4. Chain all steps together to prevent leakage

In [ ]:
# Numeric preprocessing: impute + scale
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Categorical preprocessing: impute + encode
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Combine transformers
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='drop'  # Drop any columns not specified
)

print("✓ Preprocessor pipeline created!")
print(f"\nNumeric features: {len(numeric_features)}")
print(f"Categorical features: {len(categorical_features)}")

### 4.3 Full Pipeline: Preprocessing + Model

In [ ]:
# Create full pipeline: preprocessing + model
full_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

print("=== FULL PIPELINE ===")
print(full_pipeline)

# Fit on training data ONLY
print("\nFitting pipeline on training data...")
full_pipeline.fit(X_train, y_train)
print("✓ Pipeline fitted!")

# Evaluate on validation set
train_score = full_pipeline.score(X_train, y_train)
val_score = full_pipeline.score(X_val, y_val)

print(f"\n=== SCORES ===")
print(f"Train R²: {train_score:.4f}")
print(f"Validation R²: {val_score:.4f}")
print(f"Difference: {train_score - val_score:.4f}")

### 4.4 Inspecting the Pipeline

Understanding what features are created:

In [ ]:
# Get feature names after transformation
try:
    feature_names = full_pipeline.named_steps['preprocessor'].get_feature_names_out()
    print(f"=== TRANSFORMED FEATURES ===")
    print(f"Total features after preprocessing: {len(feature_names)}")
    print(f"\nFirst 10 features:")
    for i, name in enumerate(feature_names[:10], 1):
        print(f"  {i}. {name}")
    
    if len(feature_names) > 10:
        print(f"  ... and {len(feature_names) - 10} more")
except:
    print("Feature names not available (older sklearn version)")
    print(f"Estimated features: {len(numeric_features)} numeric features (scaled)")

## 📝 PAUSE-AND-DO Exercise 2 (10 minutes)

**Task:** Create a full sklearn Pipeline and run one validation score.

**Instructions:**
1. The pipeline is already created above
2. Modify the pipeline to try different imputation strategies:
   - Change median to mean for numeric features
   - Try different imputation for categorical features
3. Compare the validation scores
4. Document your findings below

---

In [ ]:
# YOUR CODE HERE: Experiment with different preprocessing strategies

# Example: Try mean imputation instead of median
numeric_transformer_v2 = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # Changed from median
    ('scaler', StandardScaler())
])

preprocessor_v2 = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer_v2, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='drop'
)

full_pipeline_v2 = Pipeline(steps=[
    ('preprocessor', preprocessor_v2),
    ('regressor', LinearRegression())
])

# Fit and evaluate
full_pipeline_v2.fit(X_train, y_train)
train_score_v2 = full_pipeline_v2.score(X_train, y_train)
val_score_v2 = full_pipeline_v2.score(X_val, y_val)

print("=== COMPARISON ===")
print(f"\nOriginal (median imputation):")
print(f"  Train R²: {train_score:.4f}")
print(f"  Val R²: {val_score:.4f}")

print(f"\nAlternative (mean imputation):")
print(f"  Train R²: {train_score_v2:.4f}")
print(f"  Val R²: {val_score_v2:.4f}")

print(f"\nDifference in validation score: {val_score_v2 - val_score:.6f}")

### YOUR FINDINGS HERE:

**Finding 1:**  
[Compare the two imputation strategies]

**Finding 2:**  
[Discuss which performs better and why]

**Finding 3:**  
[Any other observations about the pipeline]

---

## 5. Gemini Prompt Cards for Pipeline Generation

### Example Prompts:

**Prompt 1: Generate Pipeline**
```
Create a scikit-learn pipeline that:
1. Imputes missing values (median for numeric, most_frequent for categorical)
2. Scales numeric features using StandardScaler
3. Encodes categorical features using OneHotEncoder
4. Fits a LinearRegression model

Use ColumnTransformer to handle different feature types separately.
```

**Prompt 2: Debug Pipeline**
```
I'm getting an error when fitting my pipeline: [error message]
My pipeline code is: [code]
Help me debug this issue and explain what's wrong.
```

**Prompt 3: Extend Pipeline**
```
I have a working pipeline with imputation and scaling.
How can I add polynomial features (degree=2) only to numeric features
while keeping the categorical encoding unchanged?
```

**Remember:** After using Gemini, always:
1. Verify the code runs without errors
2. Check the output makes sense
3. Add your own comments explaining each step

## 6. Pipeline Done Right Checklist

Before moving on, verify your pipeline meets these standards:

### ✓ Pre-fitting Checks:
- [ ] Split data BEFORE building pipeline
- [ ] Identified all numeric and categorical features
- [ ] Handled missing values with appropriate strategy
- [ ] Separate transformers for different feature types

### ✓ Fitting Checks:
- [ ] Pipeline fitted ONLY on training data
- [ ] No data leakage from validation or test sets
- [ ] All preprocessing steps are inside the pipeline
- [ ] Model is the last step in the pipeline

### ✓ Evaluation Checks:
- [ ] Evaluated on validation set (not test set)
- [ ] Training and validation scores are reasonable
- [ ] No major signs of overfitting (huge train/val gap)
- [ ] Pipeline can transform new data without refitting

### ✓ Code Quality:
- [ ] All parameters are explicit (no hidden defaults)
- [ ] Steps are named clearly
- [ ] Comments explain "why" not just "what"
- [ ] Can explain every step if asked

## 7. Wrap-Up: Key Takeaways

### What We Learned Today:

1. **Data Auditing**: Systematic checks for types, missingness, and unique values
2. **Preprocessing Patterns**: Separate handling for numeric vs categorical features
3. **Pipeline Architecture**: ColumnTransformer + Pipeline prevents leakage
4. **Fit/Transform Discipline**: Fit on train, transform on train/val/test
5. **Gemini Integration**: Use AI to draft, then verify and document

### Critical Rules:

> **"Fit ONLY on training data"**  
> Any statistics (mean, median, categories) must be computed from training data only.

> **"Pipeline wraps everything"**  
> If you do it manually, you risk leakage. Put it in the pipeline.

### Next Steps:

- The next notebook will cover regression metrics and baseline models
- We'll use today's pipeline structure for all future models
- Start thinking about your project dataset

---

## 8. Submission Instructions

### To Submit This Notebook:

1. **Run All Cells**: Execute `Runtime → Run all` to ensure everything works
2. **Save a Copy**: `File → Save a copy in Drive`
3. **Get Shareable Link**: Click `Share` and set to "Anyone with the link can view"
4. **Submit Link**: Paste the link in the LMS assignment

### Before Submitting, Check:

- [ ] All cells execute without errors
- [ ] All outputs are visible
- [ ] Exercise responses are complete
- [ ] Pipeline checklist is verified
- [ ] Notebook is shared with correct permissions

---

## Bibliography

- scikit-learn User Guide: [Pipelines and composite estimators](https://scikit-learn.org/stable/modules/compose.html)
- scikit-learn User Guide: [ColumnTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html)
- scikit-learn User Guide: [Preprocessing data](https://scikit-learn.org/stable/modules/preprocessing.html)
- Pedregosa et al. (2011). "Scikit-learn: Machine Learning in Python." *Journal of Machine Learning Research*, 12, 2825-2830.
- James, G., Witten, D., Hastie, T., & Tibshirani, R. (2021). *An Introduction to Statistical Learning with Python* (ISLP). Springer.

---



<center>

Thank you!

</center>